In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110259

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,182811.9452
2018-02-28,158139.2143
2018-03-31,192403.6204
2018-04-30,182853.4610
2018-05-31,184170.8338
2018-06-30,190689.8316
2018-07-31,249579.7007
2018-08-31,258714.9936
2018-09-30,241980.4443


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    182811.9452
 2018-02-28    158139.2143
 2018-03-31    192403.6204
 2018-04-30    182853.4610
 2018-05-31    184170.8338
 2018-06-30    190689.8316
 2018-07-31    249579.7007
 2018-08-31    258714.9936
 2018-09-30    241980.4443
 2018-10-31    203969.1002
 2018-11-30    194432.0165
 2018-12-31    157556.6607
 2019-01-31    249244.8544
 2019-02-28    185759.7515
 2019-03-31    298373.0318
 2019-04-30    270894.1973
 2019-05-31    254101.6994
 2019-06-30    268492.2195
 2019-07-31    302224.4093
 2019-08-31    353527.6520
 2019-09-30    257119.3183
 2019-10-31    241499.9142
 2019-11-30    241632.7563
 2019-12-31    331833.6450
 2020-01-31    257290.2941
 2020-02-29    231366.7576
 2020-03-31    234897.9855
 2020-04-30    232001.5572
 2020-05-31    192218.2425
 2020-06-30    225825.7280
 2020-07-31    309763.5511
 2020-08-31    324019.7540
 2020-09-30    181766.6411
 2020-10-31    227290.8067
 2020-11-30    219160.5891
 2020-12-31    224788.2176

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.894181
p-value : 0.002078
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1028.955, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1024.158, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1023.989, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1023.730, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1023.823, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1022.306, Time=0.08 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1021.497, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1023.461, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1025.728, Time=0.09 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1026.524, Time=0.16 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1024.151, Time=0.13 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1023.381, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1025.328, Time=0.10 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1027.333, Time=0.17 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1027.225, Time=0.17 sec
 ARIMA(3,0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -507.753
Method:                       css-mle   S.D. of innovations          42947.874
Date:                Sat, 11 Sep 2021   AIC                           1021.507
Time:                        03:55:23   BIC                           1026.720
Sample:                             0   HQIC                          1023.417
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.302e+05   1.18e+04     19.490      0.000    2.07e+05    2.53e+05
ar.L1.y        0.4493      0.137      3.279      0.001       0.181       0.718
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([220062.20890633, 225628.43251625]), array([42947.87445281, 47083.20278707]), array([[135885.92176627, 304238.49604639],
       [133347.05077679, 317909.81425571]]))
